<a href="https://colab.research.google.com/github/jtaquia/CURSO-ANALITICA-PREDICTIVA/blob/Clasificacion-supervisada/Caso_Clasificacion_Binaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')


In [0]:
data=pd.read_csv('train.csv')
data.head()

In [0]:
data.isnull().sum() #checking for total null values

data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.')

    pd.crosstab(data.Initial,data.Sex).T 

data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
data.groupby('Initial')['Age'].mean() #lets check the average age by Initials

## Assigning the NaN Values with the Ceil values of the mean ages
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

data.Age.isnull().any() #So no null values left finally 

In [0]:
print('Highest Fare was:',data['Fare'].max())
print('Lowest Fare was:',data['Fare'].min())
print('Average Fare was:',data['Fare'].mean())

In [0]:
newlist =pd.DataFrame( [round(x) for x in data['Fare']])

hstack = pd.concat([data, newlist], axis=1)

In [0]:
hstack.rename(columns={'0': 'Rounded'}) #, inplace=True)
hstack.head()

In [0]:
data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4
data.head(2)

In [0]:
data['Age_band'].value_counts().to_frame()#.style.background_gradient(cmap='summer')#checking the number of passenegers in each band

In [0]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']#family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1#Alone

data['Fare_Range']=pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame()

In [0]:
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

In [0]:
data.isnull().sum() #checking for total null values
del data['Cabin']
data = data.dropna(how='any',axis=0) 
data.shape

In [0]:
train,test=train_test_split(data,test_size=0.3,random_state=0,stratify=data['Survived'])
train.isnull().sum() #checking for total null values
test.isnull().sum() #checking for total null values


In [0]:
train.shape
test.shape


train.head(5)

del train['Name']
del train['PassengerId']
del train['Ticket']
del train['Fare_Range']

del test['Name']
del test['PassengerId']
del test['Ticket']
del test['Fare_Range']

train_X = train[train.columns[1:]]
train_X.shape
train_X

In [0]:
print(train.describe())
print(train.columns)
train.shape
test.shape

In [0]:
mapping = {'Mr': 1, 'Mrs': 2,'Master':3,'Miss':4,'Other':5}
mapping2 ={'S':1,'C':2,'Q':3}
mapping3 ={'male':1,'female':2}


In [0]:
a_x2 = train_X.replace({'Initial': mapping,'Embarked':mapping2,'Sex':mapping3}).values
a_x2.shape
a_x2
numrows = len(a_x2)    #
numrows
print(type(a_x2))

In [0]:
train_Y=train[train.columns[0]]
print(type(train_Y))#<class 'pandas.core.series.Series'>
train_Y.shape
a_y2 = train_Y.values
a_y2.shape
print(type(a_y2)) #<class 'numpy.ndarray'>
a_y2.shape


In [0]:
model = LogisticRegression()
model.fit(a_x2,a_y2)


In [0]:
test.shape

In [0]:
test_X=test[test.columns[1:]]
test_X.head(5)
test_X.shape
#test.shape
#test.head(5)


In [0]:
b_x2 = test_X.replace({'Initial': mapping,'Embarked':mapping2,'Sex':mapping3}).values
b_x2.shape
print(type(b_x2))#<class 'numpy.ndarray'>


test_Y=test[test.columns[0]]
print(type(test_Y)) #<class 'pandas.core.series.Series'>


test_Y
b_y2 = test_Y.values
print(type(b_y2)) #<class 'numpy.ndarray'>
numrows = len(b_y2)    #
numrows

In [0]:
prediction1=model.predict(b_x2)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction1,b_y2))

In [0]:
model2=DecisionTreeClassifier()
model2.fit(a_x2,a_y2)
prediction2=model2.predict(b_x2)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction2,b_y2))


In [0]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]
classifiers=['Logistic Regression']
models=[LogisticRegression()]
for i in models:
    model = i
    cv_result = cross_val_score(model,b_x2,b_y2, cv = kfold,scoring = "accuracy")
    cv_result=cv_result
    print(cv_result)
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
new_models_dataframe2

In [0]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy,index=[classifiers])
box.T.boxplot()